## A taste of MLJ

### Baby steps

Let's load a reduced version of the well-known Ames House Price data set (containing six of the more important categorical features and six of the more important numerical features):

In [1]:
using MLJ
using DataFrames, Statistics

task = load_reduced_ames()
Xtable, y = X_and_y(task)

train, test = partition(eachindex(y), 0.70, shuffle=true); # 70:30 split

`Xtable` is a named-tuple of vectors but we can work with any tabular format supported by Tables.jl. Let's use `DataFrame`s:

In [2]:
X = DataFrame(Xtable);
first(X, 4)

,OverallQual,GrLivArea,Neighborhood,x1stFlrSF,TotalBsmtSF,BsmtFinSF1,LotArea,GarageCars,MSSubClass,GarageArea,YearRemodAdd,YearBuilt
,Float64,Float64,Categorical…,Float64,Float64,Float64,Float64,Float64,Categorical…,Float64,Float64,Float64
1,5.0,816.0,Mitchel,816.0,816.0,816.0,6600.0,2.0,_20,816.0,2003.0,1982.0
2,8.0,2028.0,Timber,2028.0,1868.0,1460.0,11443.0,3.0,_20,880.0,2006.0,2005.0
3,7.0,1509.0,Gilbert,807.0,783.0,0.0,7875.0,2.0,_60,393.0,2003.0,2003.0
4,6.0,1686.0,NWAmes,1686.0,1686.0,625.0,10240.0,2.0,_20,612.0,1980.0,1980.0


A *model* is a container for hyperparameters:

In [3]:
constant_model = ConstantRegressor()

# ConstantRegressor @ 8…79: 
target_type             =>   Float64
distribution_type       =>   Distributions.Normal{Float64}



Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [4]:
constant = machine(constant_model, X, y)

# Machine{ConstantRegressor{Float6…} @ 2…47: 
model                   =>   ConstantRegressor @ 8…79
fitresult               =>   (undefined)
cache                   =>   (undefined)
args                    =>   (omitted Tuple{DataFrame,Array{Float64,1}} of length 2)
report                  =>   empty Dict{Symbol,Any}
rows                    =>   (undefined)



Training and making a new prediction:

In [5]:
fit!(constant, rows=train)
yhat = predict(constant, X[test,:]);
yhat[1:4]

┌ Info: Training Machine{ConstantRegressor{Float6…} @ 2…47.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
┌ Info: Fitted a constant probability distribution, Distributions.Normal{Float64}(μ=180777.47595682042, σ=78131.4449300276).
└ @ MLJ.Constant /Users/anthony/Dropbox/Julia7/MLJ/src/builtins/Constant.jl:46


4-element Array{Distributions.Normal{Float64},1}:
 Distributions.Normal{Float64}(μ=180777.47595682042, σ=78131.4449300276)
 Distributions.Normal{Float64}(μ=180777.47595682042, σ=78131.4449300276)
 Distributions.Normal{Float64}(μ=180777.47595682042, σ=78131.4449300276)
 Distributions.Normal{Float64}(μ=180777.47595682042, σ=78131.4449300276)

The model we chose can provide probabilistic predictions and so does so by default.

In [6]:
mean.(yhat)[1:4]

4-element Array{Float64,1}:
 180777.47595682042
 180777.47595682042
 180777.47595682042
 180777.47595682042

In [7]:
rmsl(y[test], yhat) # applies root-mean-square-log loss to prediction means

0.38989267684583123

Or, we could have skipped the train/test definitions and evaluated one line:

In [8]:
evaluate!(constant, resampling=Holdout(fraction_train=0.7), measure=rmsl)

0.4095126492179508

### Something fancier

Let's define a composite model type that:

- one-hot encodes the inputs
- log transforms the target
- fits specified K-nearest neighbour and ridge regressor models to the data
- computes a weighted average of individual model predictions
- inverse transforms (exponentiates) the blended predictions

MLJ will eventually have macros for quickly building this kind of composite model (and more sophisticated model stacks). Nevertheless, building the model with our bare hands is easier than in other popular frameworks:

In [17]:
# the new model struture, with other models as hyperparameters:
mutable struct KNNRidgeBlend <:Deterministic{Node}
    knn_model
    ridge_model
    weight # between 0 and 1
end

In [10]:
# model fitting instructions:
function MLJ.fit(model::KNNRidgeBlend, X, y)
    
    # source nodes of the "learning network" being wrappped:    
    
    Xs = source(X) 
    ys = source(y)

    hot = machine(OneHotEncoder(), Xs)

    # W, z, zhat and yhat are nodes in the network:
    
    W = transform(hot, Xs) # one-hot encode the input
    z = log(ys) # transform the target
    
    ridge_model = model.ridge_model
    knn_model = model.knn_model

    ridge = machine(ridge_model, W, z) 
    knn = machine(knn_model, W, z)

    zhat = model.weight*predict(ridge, W) + (1 - model.weight)*predict(knn, W) # average the predictions of KNN and ridge models
    yhat = exp(zhat) # inverse the target transformation
    
    fit!(yhat, verbosity=0)
    
    return yhat
end


We now instantiate a blended model and evaluate it:

In [11]:
knn_model = KNNRegressor(K=2)
ridge_model = RidgeRegressor(lambda=0.1)
knn_ridge_blend_model = KNNRidgeBlend(knn_model, ridge_model, 0.90)
knn_ridge_blend = machine(knn_ridge_blend_model, X, y)
er = evaluate!(knn_ridge_blend, resampling=Holdout(fraction_train=0.7), measure=rmsl) 

0.13108966715891057

We can inspect the nested hyperparameters of our composite model:

In [12]:
params(knn_ridge_blend_model)

Params(:knn_model => Params(:target_type => Float64, :K => 2, :metric => MLJ.KNN.euclidean, :kernel => MLJ.KNN.reciprocal), :ridge_model => Params(:target_type => Float64, :lambda => 0.1), :weight => 0.9)

For our next trick, we wrap the model in a tuning strategy, producing a new tuned model. 

The form of `nested_ranges` below matches the pattern above (with parameters not being tuned omitted):

In [13]:
K_range = range(knn_model, :K, lower=2, upper=100, scale=:log10)
lambda_range = range(ridge_model, :lambda, lower= 1e-6, upper=10, scale = :log10)

nested_ranges = Params(:knn_model => Params(:K => K_range), :ridge_model => Params(:lambda => lambda_range))

tuning = Grid(resolution=12)
resampling = CV(nfolds=6)

tuned_blended_model = TunedModel(model=knn_ridge_blend_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges, measure=rmsl)

# DeterministicTunedModel @ 9…32: 
model                   =>   KNNRidgeBlend @ 3…35
tuning                  =>   Grid @ 1…77
resampling              =>   CV @ 8…17
measure                 =>   rmsl (generic function with 2 methods)
operation               =>   predict (generic function with 18 methods)
nested_ranges           =>   Params(:knn_model => Params(:K => NumericRange @ 1…46), :ridge_model => Params(:lambda => NumericRange @ 1…88))
report_measurements     =>   true



Fitting a corresponding machine searches for the model with optimal performance, as determined by the specified tuning/resampling strategy, and trains the best model on all specifed data (in this case on `train`). So, predictions of the fitted machine are predictions using optimal hyperparameters.

In [14]:
tuned_blended = machine(tuned_blended_model, X, y)
fit!(tuned_blended, rows=train);

┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…26.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:69
Searching a 144-point grid for best model: 100%[=========================] Time: 0:01:29
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:130


In [15]:
best_model = tuned_blended.report[:best_model]
@show best_model.knn_model.K;
@show best_model.ridge_model.lambda;

(best_model.knn_model).K = 8
(best_model.ridge_model).lambda = 2.3101297000831598


In [16]:
best = machine(best_model, X, y)
evaluate!(best, resampling=Holdout(fraction_train=0.7), measure=rmsl)

0.1306583597860082